In [ ]:
# BUSINESS CHALLENGE:
# Create a product that builds a Brochure for a company to be used for prospective clients, 
#  investors and potential recruits.

In [2]:
from openai import OpenAI
import os 
import requests 
import json 
from typing import List 
from dotenv import load_dotenv
from bs4 import BeautifulSoup 
from IPython.display import Markdown, display, update_display 



In [33]:
IS_LOCAL=True
apikey = ""
MODEL = ""


if(IS_LOCAL):
    !ollama pull llama3.2
    MODEL = "llama3.2"
    openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama") 
else:
    load_dotenv(override=True)
    MODEL="gpt-4o-mini"
    openai = OpenAI()
    apikey = os.getenv('OPENAI_API_KEY')
    if(apikey):
       print("Api key looks good so far!")
    else:
      print("Not found OPENAI_API_KEY!")


pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [34]:

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
     A utility class to represent a Website that we have scraped, now with links
    """
    def __init__(self, url):
        self.url = url 
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")      
        self.title = soup.title.string  if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else: 
            self.text = ""
        links = [link.get("href") for link in soup.find_all('a')]      
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title: \n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    




In [35]:
link_system_prompt = """You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in
a brochure about the company, such as links to an About page, or a Company page,  
or Careers/Jobs pages. \n 
"""
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
    you should respond in JSON as in this example:
"""
link_system_prompt += """
{
   "links": [ 
       { "type": "about page", "url": "https://full.url/goes/here/about"}, 
       { "type": "careers page", "url": "https://another.full.url/careers"}
   ]
}

"""


In [36]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += """ please decide wich of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links
: \n    """
    user_prompt += "Links (some might be relative links): \n"
    user_prompt += "\n".join(website.links)
    return user_prompt



In [37]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(website)}
        ],
         response_format={"type": "json_object"}
      
    )
    result = response.choices[0].message.content
    return json.loads(result)
  

In [38]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
 
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        if link['url']:
            result += Website(link["url"]).get_contents()
    return result

In [39]:
system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website 
and creates a short brochure about the company for prospective customers, investors and
recruits.  Respond in Markdown.
Include details of company culture, customers and careers/jobs if you have the information.

"""

In [40]:
def get_brochure_user_prompt(company_name, url):
        user_prompt = f"ypu are looking at a company called: {company_name} \n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown. \n"
        user_prompt +=  get_all_details(url)
        user_prompt = user_prompt[:5_000]
        return user_prompt

In [41]:
get_brochure_user_prompt("IFOOD",  "https://www.ifood.com.br/")

TypeError: string indices must be integers

In [ ]:
def create_brochure(company_name, url):
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
            ]
        )
        result = response.choices[0].message.content
        display(Markdown(result))
    

In [ ]:
create_brochure("IFOOD", "https://www.ifood.com.br")